In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio


def generate_sinusoidal_waveform(freq=2, duration=5, sample_rate=8000):
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    waveform = np.sin(2 * np.pi * freq * t).reshape(1, -1)
    return waveform, sample_rate


# Wizualizacja sygnału
def plot_waveform(waveform):
    plt.figure(figsize=(10, 4))
    plt.plot(waveform[0])
    plt.title("Audio Signal")
    plt.xlabel("Sample Index")
    plt.ylabel("Amplitude")
    plt.show()


def plot_spectrogram(waveform, sample_rate):
    plt.figure(figsize=(10, 4))
    plt.specgram(waveform[0], Fs=sample_rate, scale="default", mode="magnitude")
    plt.title("Spectrogram")
    plt.xlabel("Time")
    plt.ylabel("Frequency")
    plt.colorbar()
    plt.show()


# Dodawanie szumu Poissona
def add_scaled_noise(
    signal: np.ndarray, intensity=0.5, noise_type="gaussian"
) -> np.ndarray:
    rms_signal = np.sqrt(np.mean(signal**2))
    rms_noise = intensity * rms_signal

    match noise_type:
        case "gaussian":
            noise = np.random.normal(0, rms_noise, signal.shape)
        case "rayleigh":
            sigma = rms_noise / np.sqrt(2 - np.pi / 2)
            noise = np.random.rayleigh(sigma, signal.shape)
            noise = noise - np.mean(noise) 
            signs = np.sign(signal)
            noise = noise * rms_noise / np.sqrt(np.mean(noise**2))
            noise = noise * signs
        case "poisson":
            lambda_vals = np.abs(signal) * rms_noise
            noise = np.random.poisson(lambda_vals, signal.shape) - lambda_vals
            noise = noise - np.mean(noise) 
            signs = np.sign(signal)
            noise = noise * rms_noise / np.sqrt(np.mean(noise**2))
            noise = noise * signs
        case _:
            raise ValueError("Nieobsługiwany typ szumu.")

    return signal + noise


waveform, sample_rate = generate_sinusoidal_waveform()
plot_waveform(waveform)
plot_spectrogram(waveform, sample_rate)
display(Audio(waveform[0], rate=sample_rate))

noisy_waveform = add_scaled_noise(waveform, intensity=1, noise_type="gaussian")
plot_waveform(noisy_waveform)
plot_spectrogram(noisy_waveform, sample_rate)
display(Audio(noisy_waveform[0], rate=sample_rate))

noisy_waveform = add_scaled_noise(waveform, intensity=1, noise_type="rayleigh")
plot_waveform(noisy_waveform)
plot_spectrogram(noisy_waveform, sample_rate)
display(Audio(noisy_waveform[0], rate=sample_rate))

noisy_waveform = add_scaled_noise(waveform, intensity=1, noise_type="poisson")
plot_waveform(noisy_waveform)
plot_spectrogram(noisy_waveform, sample_rate)
display(Audio(noisy_waveform[0], rate=sample_rate))
